In [ ]:
%load_ext nb_black
%load_ext autoreload

%autoreload 2

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np

np.random.seed(0)


import os
from requests import get
from pathlib import Path
import gc

from matplotlib import pyplot as plt
from zipfile import ZipFile

%matplotlib inline

In [ ]:
from tensorflow.keras.utils import plot_model

In [ ]:
from thc_net.classifier import ThcNetClassifier
from thc_net.utils import download, plot_history
from thc_net.input_utils import prepare_input_data

# Bank marketing  : loading data

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip"
dataset_name = "bank-additional-full"
out_zip = Path(os.getcwd().rsplit("/", 1)[0] + "/data/bank-additional.zip")
out = Path(
    os.getcwd().rsplit("/", 1)[0] + "/data/bank-additional/" + dataset_name + ".csv"
)

download(url, out_zip)
with ZipFile(out_zip, "r") as zipObj:
    zipObj.extractall("/data")

target = "y"
to_remove = []
train = pd.read_csv(out, sep=";", low_memory=False)

# Census income  : loading data

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
dataset_name = "census-income"
out = Path(os.getcwd().rsplit("/", 1)[0] + "/data/" + dataset_name + ".csv")


download(url, out)
target = " <=50K"
to_remove = []
train = pd.read_csv(out, sep=",", low_memory=False)

# Load data and split

In [ ]:
if "Set" not in train.columns:
    train["Set"] = np.random.choice(
        ["train", "valid", "test"], p=[0.8, 0.1, 0.1], size=(train.shape[0],)
    )

train_indices = train[train.Set == "train"].index
valid_indices = train[train.Set == "valid"].index
test_indices = train[train.Set == "test"].index

In [ ]:
Y = train[target].values
X = train.drop(columns=["Set"] + [target])

In [ ]:
ratio = 0.005

In [ ]:
n_unique = X.nunique()
ratios = (n_unique / X.shape[0]) < ratio
cat_idxs = np.argwhere(
    X.columns.isin(X.columns[ratios | (X.dtypes == "object")])
).ravel()
cat_dims = n_unique[cat_idxs].values + X.isnull().sum()[cat_idxs].values + 1
del n_unique, ratios, train

In [ ]:
gc.collect()

# Training

In [ ]:
tg_enc = LabelEncoder()
Y = tg_enc.fit_transform(Y)

In [ ]:
X_train = X.values[train_indices]
y_train = Y[train_indices]

X_valid = X.values[valid_indices]
y_valid = Y[valid_indices]

X_test = X.values[test_indices]
y_test = Y[test_indices]

In [ ]:
X_train.shape

In [ ]:
X_train_prep, encoders = prepare_input_data(X_train, cat_idxs)
X_valid_prep, _ = prepare_input_data(X_valid, cat_idxs, encoders=encoders, fit=False)
X_test_prep, _ = prepare_input_data(X_test, cat_idxs, encoders=encoders, fit=False)

# Network parameters

In [ ]:
metrics = ["AUC"]

In [ ]:
%%time

model = ThcNetClassifier(
    n_layer=3,
    mul_input=8, 
    metrics=metrics,
    cat_idxs=cat_idxs,
    cat_emb_dims=cat_dims,
    dropout=0.05,
    normalize=False,
    max_emb=20,
    patience=30
)

history = model.fit(
        X=X_train_prep, 
        y=y_train,
        X_valid=X_valid_prep,
        y_valid=y_valid,
        batch_size=1024,
        epochs=10000,
        verbose=2,
)

In [ ]:
%%time

model_snn = ThcNetClassifier(
    n_layer=3,
    mul_input=8, 
    metrics=metrics,
    cat_idxs=cat_idxs,
    cat_emb_dims=cat_dims,
    dropout=0.05,
    normalize=False,
    max_emb=20,
    patience=30,
    use_snn=True,
    noise=None
)

history_snn = model_snn.fit(
        X=X_train_prep, 
        y=y_train,
        X_valid=X_valid_prep,
        y_valid=y_valid,
        batch_size=1024,
        epochs=10000,
        verbose=2,
)

In [ ]:
model.network.summary()

In [ ]:
plot_model(
    model,
    # to_file="model.png",
    show_shapes=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
)

In [ ]:
model_snn.network.summary()

### Predictions

In [ ]:
plot_history(history)

In [ ]:
plot_history(history_snn)

In [ ]:
y_pred = model.predict_proba(X_test_prep)
test_auc = roc_auc_score(y_score=y_pred[:, 1], y_true=y_test)
print(f"FINAL TEST SCORE FOR {dataset_name} : {test_auc}")

In [ ]:
y_pred = model_snn.predict_proba(X_test_prep)
test_auc = roc_auc_score(y_score=y_pred[:, 1], y_true=y_test)
print(f"FINAL TEST SCORE FOR {dataset_name} : {test_auc}")